# Business Problem 
### As a food blogger visiting Toronto city, I want to stay at a hotel such that my travelling distance in minimal to all restraurents in the city


## 

### Load required Libraries

In [1]:
import pandas as pd
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import requests

In [2]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Get Lat-Long for postal codes

In [3]:
geo_ng_data = pd.read_csv('https://cocl.us/Geospatial_data')

In [4]:
geo_ng_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [5]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
    
# add markers to map
for lat, lng in zip(geo_ng_data['Latitude'], geo_ng_data['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [6]:
CLIENT_ID = 'DFETSOEBU2Y55YH5KJNGZKY4FLVRWD2SK0ZSUDPHZDU4XWRR' # your Foursquare ID
CLIENT_SECRET = 'B4LLOGTOAOTOHFAQZDIYB3HR5IAPOI0VANGJXT2GFZUACSA5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DFETSOEBU2Y55YH5KJNGZKY4FLVRWD2SK0ZSUDPHZDU4XWRR
CLIENT_SECRET:B4LLOGTOAOTOHFAQZDIYB3HR5IAPOI0VANGJXT2GFZUACSA5


# Get venues in Toronto using foursquare api

In [7]:
def getNearbyVenues(latitudes, longitudes, radius=300):
    venues_list=[]
    for lat, lng in zip(latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
toronto_venues = getNearbyVenues(
    latitudes=geo_ng_data['Latitude'],
    longitudes=geo_ng_data['Longitude']
)

In [9]:
toronto_hotel = toronto_venues[toronto_venues['Venue Category'] == 'Hotel'].reset_index(drop=True)
toronto_hotel['Color'] = '#ff0000'
toronto_hotel['Key'] = 0

In [10]:
print(toronto_hotel.shape)
toronto_hotel

(17, 6)


,Venue,Venue Latitude,Venue Longitude,Venue Category,Color,Key
0,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0
1,The Saint James Hotel,43.659398,-79.380932,Hotel,#ff0000,0
2,Pantages Hotel & Spa,43.654498,-79.379035,Hotel,#ff0000,0
3,The Omni King Edward Hotel,43.649191,-79.376006,Hotel,#ff0000,0
4,Shangri-La Toronto,43.649129,-79.386557,Hotel,#ff0000,0
5,Hilton,43.649946,-79.385479,Hotel,#ff0000,0
6,Delta Hotels by Marriott Toronto,43.642882,-79.383949,Hotel,#ff0000,0
7,Le Germain Hotel,43.643125,-79.380918,Hotel,#ff0000,0
8,The Fairmont Royal York,43.645449,-79.381508,Hotel,#ff0000,0
9,Cosmopolitan Toronto Centre Hotel & Spa,43.649064,-79.377598,Hotel,#ff0000,0


In [11]:
toronto_restaurant = toronto_venues[toronto_venues['Venue Category'].str.contains('Restaurant')]
toronto_restaurant['Venue Category'] = 'Restaurant'
toronto_restaurant['Color'] = '#8000ff'
toronto_restaurant['Key'] = 0

<ipython-input-11-6fbbc4d4c52a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toronto_restaurant['Venue Category'] = 'Restaurant'
<ipython-input-11-6fbbc4d4c52a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toronto_restaurant['Color'] = '#8000ff'
<ipython-input-11-6fbbc4d4c52a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [12]:
print(toronto_restaurant.shape)
toronto_restaurant

(275, 6)


,Venue,Venue Latitude,Venue Longitude,Venue Category,Color,Key
1,Korean Grill House,43.770812,-79.214502,Restaurant,#8000ff,0
2,Federick Restaurant,43.774697,-79.241142,Restaurant,#8000ff,0
3,Drupati's Roti & Doubles,43.775222,-79.241678,Restaurant,#8000ff,0
4,Thai One On,43.774468,-79.241268,Restaurant,#8000ff,0
10,Vincent's Spot,43.717002,-79.242353,Restaurant,#8000ff,0
...,...,...,...,...,...,...
1109,Tosto,43.661198,-79.386414,Restaurant,#8000ff,0
1112,Teriyaki Experience,43.659884,-79.387879,Restaurant,#8000ff,0
1115,Great North American Grill,43.638429,-79.618756,Restaurant,#8000ff,0
1118,Chick-n-Joy,43.665181,-79.321403,Restaurant,#8000ff,0


In [13]:
toronto_hotel_restaurant = toronto_hotel.append(toronto_restaurant).reset_index(drop=True)

In [14]:
toronto_hotel_restaurant

,Venue,Venue Latitude,Venue Longitude,Venue Category,Color,Key
0,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0
1,The Saint James Hotel,43.659398,-79.380932,Hotel,#ff0000,0
2,Pantages Hotel & Spa,43.654498,-79.379035,Hotel,#ff0000,0
3,The Omni King Edward Hotel,43.649191,-79.376006,Hotel,#ff0000,0
4,Shangri-La Toronto,43.649129,-79.386557,Hotel,#ff0000,0
...,...,...,...,...,...,...
287,Tosto,43.661198,-79.386414,Restaurant,#8000ff,0
288,Teriyaki Experience,43.659884,-79.387879,Restaurant,#8000ff,0
289,Great North American Grill,43.638429,-79.618756,Restaurant,#8000ff,0
290,Chick-n-Joy,43.665181,-79.321403,Restaurant,#8000ff,0


# Represent hotels by Red color marker

In [15]:
# create map
map_data = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to the map
markers_colors = []
for lat, lon, poi,clusterName,colorV in zip(toronto_hotel_restaurant['Venue Latitude'],
                                           toronto_hotel_restaurant['Venue Longitude'],
                                           toronto_hotel_restaurant['Venue'],
                                           toronto_hotel_restaurant['Venue Category'],
                                          toronto_hotel_restaurant['Color']
                                          ):
    label = folium.Popup(str(clusterName) + '-' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colorV,
        fill=True,
        fill_color=colorV,
        fill_opacity=0.7).add_to(map_data)
       
map_data

In [16]:
from math import sin, cos, sqrt, atan2, radians

In [17]:
def getDistance(lat1, lon1, lat2, lon2):
    
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c

    return distance

In [19]:
lat1 = 52.2296756
lon1 = 21.0122287
lat2 = 52.406374
lon2 = 16.9251681
dist1 = getDistance(lat1, lon1, lat2, lon2)

dist1

278.54558935106695

In [20]:
toronto_hotel_restaurant_dist = pd.merge(toronto_hotel,toronto_restaurant, how='left',on='Key')
toronto_hotel_restaurant_dist

,Venue_x,Venue Latitude_x,Venue Longitude_x,Venue Category_x,Color_x,Key,Venue_y,Venue Latitude_y,Venue Longitude_y,Venue Category_y,Color_y
0,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Korean Grill House,43.770812,-79.214502,Restaurant,#8000ff
1,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Federick Restaurant,43.774697,-79.241142,Restaurant,#8000ff
2,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Drupati's Roti & Doubles,43.775222,-79.241678,Restaurant,#8000ff
3,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Thai One On,43.774468,-79.241268,Restaurant,#8000ff
4,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Vincent's Spot,43.717002,-79.242353,Restaurant,#8000ff
...,...,...,...,...,...,...,...,...,...,...,...
4670,Hilton Garden Inn,43.638519,-79.618721,Hotel,#ff0000,0,Tosto,43.661198,-79.386414,Restaurant,#8000ff
4671,Hilton Garden Inn,43.638519,-79.618721,Hotel,#ff0000,0,Teriyaki Experience,43.659884,-79.387879,Restaurant,#8000ff
4672,Hilton Garden Inn,43.638519,-79.618721,Hotel,#ff0000,0,Great North American Grill,43.638429,-79.618756,Restaurant,#8000ff
4673,Hilton Garden Inn,43.638519,-79.618721,Hotel,#ff0000,0,Chick-n-Joy,43.665181,-79.321403,Restaurant,#8000ff


In [21]:
Dist = []

In [22]:
for i, rowi in toronto_hotel_restaurant_dist.iterrows():
           
        lat1 = rowi['Venue Latitude_x']
        lon1 = rowi['Venue Longitude_x']
        lat2 = rowi['Venue Latitude_y']
        lon2 = rowi['Venue Longitude_y']
        
        dist = getDistance(lat1, lon1, lat2, lon2)
        Dist.append(dist)
toronto_hotel_restaurant_dist['Dist'] = Dist

# Build Hotel-Restraurent Distance (in KM) Matrix

In [23]:
toronto_hotel_restaurant_dist

,Venue_x,Venue Latitude_x,Venue Longitude_x,Venue Category_x,Color_x,Key,Venue_y,Venue Latitude_y,Venue Longitude_y,Venue Category_y,Color_y,Dist
0,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Korean Grill House,43.770812,-79.214502,Restaurant,#8000ff,13.995952
1,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Federick Restaurant,43.774697,-79.241142,Restaurant,#8000ff,13.290812
2,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Drupati's Roti & Doubles,43.775222,-79.241678,Restaurant,#8000ff,13.324828
3,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Thai One On,43.774468,-79.241268,Restaurant,#8000ff,13.263485
4,Days Inn,43.667145,-79.313179,Hotel,#ff0000,0,Vincent's Spot,43.717002,-79.242353,Restaurant,#8000ff,7.949833
...,...,...,...,...,...,...,...,...,...,...,...,...
4670,Hilton Garden Inn,43.638519,-79.618721,Hotel,#ff0000,0,Tosto,43.661198,-79.386414,Restaurant,#8000ff,18.866108
4671,Hilton Garden Inn,43.638519,-79.618721,Hotel,#ff0000,0,Teriyaki Experience,43.659884,-79.387879,Restaurant,#8000ff,18.730374
4672,Hilton Garden Inn,43.638519,-79.618721,Hotel,#ff0000,0,Great North American Grill,43.638429,-79.618756,Restaurant,#8000ff,0.010321
4673,Hilton Garden Inn,43.638519,-79.618721,Hotel,#ff0000,0,Chick-n-Joy,43.665181,-79.321403,Restaurant,#8000ff,24.111203


In [24]:
toronto_hotel_restaurant_dist_agg = toronto_hotel_restaurant_dist.groupby(by='Venue_x').sum('Dist').sort_values(by=['Dist'])

In [25]:
toronto_hotel_restaurant_dist_agg = toronto_hotel_restaurant_dist_agg.drop(['Venue Latitude_x','Venue Longitude_x','Key','Venue Latitude_y','Venue Longitude_y'],axis=1).reset_index()

In [26]:
toronto_hotel_restaurant_dist_agg

,Venue_x,Dist
0,Pantages Hotel & Spa,1022.208863
1,The Saint James Hotel,1026.789134
2,Hilton,1042.521294
3,One King West Hotel & Residence,1054.751323
4,Cosmopolitan Toronto Centre Hotel & Spa,1057.387112
5,Shangri-La Toronto,1057.825087
6,The Omni King Edward Hotel,1068.054933
7,The Fairmont Royal York,1103.211625
8,Hotel Novotel Toronto Centre,1120.737351
9,Le Germain Hotel,1156.316850


In [27]:
toronto_optimal_hotel = toronto_hotel_restaurant_dist_agg['Venue_x'][0]

# Pantages Hotel & Spa is the recommened hotel which has minimum distance from all restraurents

In [28]:
toronto_optimal_hotel

'Pantages Hotel & Spa'

In [29]:
toronto_hotel_restaurant[toronto_hotel_restaurant['Venue'] == toronto_optimal_hotel]

,Venue,Venue Latitude,Venue Longitude,Venue Category,Color,Key
2,Pantages Hotel & Spa,43.654498,-79.379035,Hotel,#ff0000,0


In [30]:
selected_hotel_lat = toronto_hotel_restaurant[toronto_hotel_restaurant['Venue'] == toronto_optimal_hotel]['Venue Latitude']
selected_hotel_lon = toronto_hotel_restaurant[toronto_hotel_restaurant['Venue'] == toronto_optimal_hotel]['Venue Longitude']


In [31]:
print(selected_hotel_lat)

2    43.654498
Name: Venue Latitude, dtype: float64


In [32]:
toronto_hotel_restaurant['Color'] = np.where(toronto_hotel_restaurant['Venue'] == toronto_optimal_hotel,'#000000',
                                             np.where(toronto_hotel_restaurant['Venue Category'] == 'Hotel','#ff0000','#00FF00'))

In [33]:
# create map
map_data = folium.Map(location=[selected_hotel_lat, selected_hotel_lon], zoom_start=14)

# add markers to the map
markers_colors = []
for lat, lon, poi,clusterName,colorV in zip(toronto_hotel_restaurant['Venue Latitude'],
                                           toronto_hotel_restaurant['Venue Longitude'],
                                           toronto_hotel_restaurant['Venue'],
                                           toronto_hotel_restaurant['Venue Category'],
                                          toronto_hotel_restaurant['Color']
                                          ):
    label = folium.Popup(str(clusterName) + '-' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colorV,
        fill=True,
        fill_color=colorV,
        fill_opacity=0.7).add_to(map_data)
       
map_data

# Pantages Hotel & Spa is highly recommended for Food blogger (indicated as black marker on map) which has minimun distance from all restraurents